In [1]:
import tensorflow as tf
import numpy as np


In [ ]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, a convolutional, max-pooling and softmax layer.
    """
    # length of the sentences,2 classes- positive & negative, size of vocab,  dimensionality of the embedding, filter sizes for different sized words (words under a filter), number of filters per filter size
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout to feed actual data into a computational graph
        #for input data, target labels, and dropout probability
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")  #none for dynamic input sizes
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # l2 regularization loss=0 initially
        # prevents overfitting
        l2_loss = tf.constant(0.0)

        # Embedding layer
        #executed on the CPU only, group under the 'embedding' name scope
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            #W represents the embedding matrix, uniform distribution between -1.0 and 1.0 exists
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            #add new dimension at end , updated during the training process
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        # iterate over different filter sizes
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer has width of filter_size, depth is 1 because of text , output as num_filters7
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                #filter weights using a truncated normal distribution
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                # bias
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                # feature map capturing local patterns
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs , reduce the spatial dimensions of the input volume
                pooled = tf.nn.max_pool(
                    h,  #input tensor
                    ksize=[1, sequence_length - filter_size + 1, 1, 1], # size of the max pooling window or kernel, slide a k sized window
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3) # concat along the fourth dimension (axis 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total]) #change into a 2D tensor,-1 automatically infers the size of that dimension

        # Add dropout
        #regularization technique to randomly drop out (sets to zero) a fraction of the units during training to prevent overfitting
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            #Weight matrix connecting the flattened and pooled features to the output layer
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer()) # method for weight initialization
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        # softmax cross-entropy loss between the predicted scores (self.scores) and the true labels (self.input_y)
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        # mean of the correct predictions, indicating the proportion of correctly classified examples
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")